In [ ]:
nltk.download()

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

import functools # for partial parametres

In [ ]:
import gensim # td-idf

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from nltk.corpus import wordnet as wn # sentence similarity with wordnet

In [ ]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn

In [ ]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

# Пример сравнения похожести документов (Тупой, через TD-IDF статистики)

In [ ]:
raw_documents = ["I'm taking the show on the road.",
                 "My socks are a force multiplier.",
             "I am the barber who cuts everyone's hair who doesn't cut their own.",
             "Legend has it that the mind is a mad monkey.",
            "I make my own fun."]
print("Number of documents:",len(raw_documents))

In [ ]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in raw_documents]
print(gen_docs)

In [ ]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary[5])
print(dictionary.token2id['road'])
print("Number of words in dictionary:",len(dictionary))
for i in range(len(dictionary)):
    print(i, dictionary[i])

Now we will create a corpus. A corpus is a list of bags of words. A bag-of-words representation for a document just lists the number of times each word occurs in the document.

In [ ]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]
print(corpus)

Что такое TF-IDF

http://nlpx.net/archives/57

In [ ]:
tf_idf = gensim.models.TfidfModel(corpus)
print(tf_idf)
s = 0
for i in corpus:
    s += len(i)
print(s)

Now we will create a similarity measure object in tf-idf space.
tf-idf stands for term frequency-inverse document frequency. Term frequency is how often the word shows up in the document and inverse document fequency scales the value by how rare the word is in the corpus.

In [ ]:
sims = gensim.similarities.Similarity('./workdir/',tf_idf[corpus],
                                      num_features=len(dictionary))
print(sims)
print(type(sims))

Now create a query document and convert it to tf-idf.

In [ ]:
query_doc = [w.lower() for w in word_tokenize("Socks are a force for good.")]
print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
print(query_doc_tf_idf)

We show an array of document similarities to query. We see that the second document is the most similar with the overlapping of socks and force.

# Схожесть предложений с использованием Wordnet

In [ ]:
# You can check out how frequent a lemma is by doing:
cat = wn.synsets('cat', 'n')[0]     # Get the most common synset
print (cat.lemmas()[0].count()  )     # Get the first lemma => 18
 
dog = wn.synsets('dog', 'n')[0]           # Get the most common synset
feline = wn.synsets('feline', 'n')[0]     # Get the most common synset
mammal = wn.synsets('mammal', 'n')[0]     # Get the most common synset
 
# You can read more about the different types of wordnet similarity measures here: http://www.nltk.org/howto/wordnet.html
for synset in [dog, feline, mammal]:
    print ("Similarity(%s, %s) = %s" % (cat, synset, cat.wup_similarity(synset)))
 
# Similarity(Synset('cat.n.01'), Synset('dog.n.01')) = 0.2
# Similarity(Synset('cat.n.01'), Synset('feline.n.01')) = 0.5
# Similarity(Synset('cat.n.01'), Synset('mammal.n.01')) = 0.2
 

http://nlpforhackers.io/wordnet-sentence-similarity/

In [ ]:
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None
 

In [ ]:
def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(sentence1)#pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(sentence2)#pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # -----------------------------------------------------------
    
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        curr_arr = [x for x in [synset.path_similarity(ss) for ss in synsets2] if x is not None]
        curr_arr.append(0)
#         print(curr_arr)
        
        best_score = max(curr_arr)
 
        # Check that the similarity could have been computed
        if best_score is not None:
            score += best_score
            count += 1

    score /= count
    return score
 
sentences = [
    "Programming technologies",
    "Some gorgeous creatures are felines.",
    "Dolphins are swimming mammals.",
    "C++ Is beautiful shit.",
]
 
focus_sentence = "C++ are beautiful programming language"
 
for sentence in sentences:
    print("Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, sentence_similarity(focus_sentence, sentence)))
    print("Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, sentence_similarity(sentence, focus_sentence)))
 

# Технические теги

In [ ]:
technical_tags = pd.read_csv('../DB/most_popular_stack_overflow.csv')
technical_tags = set(technical_tags['TagName'])
technical_tags = {x for x in technical_tags if x==x}

technical_tags_dict = {}
technical_tags_ind_dict = {}

index = 0
for x in technical_tags:
    technical_tags_dict[x] = index
    technical_tags_ind_dict[index] = x
    index += 1

# Синонимичный ряд замен для тегов технологий

In [ ]:
technical_tags_syn = pd.read_csv('../DB/synonyms_for_stackoverflow.csv')
technical_tags_syn.head()

technical_tags_syn_dict = dict(zip(technical_tags_syn.SOurceTagName, technical_tags_syn.TargetTagName))
# technical_tags_syn.set_index('SOurceTagName')['TargetTagName'].to_dict('index')
technical_tags_syn_dict

def technical_tags_syn_try_replace(x):
    if x in technical_tags_syn_dict.keys():
        return technical_tags_syn_dict[x]
    else:
        return x

# Подобие семантической сети тегов технологий

In [ ]:
small_stackoverflow = pd.read_csv('../DB/Small_stackoverflow.csv')
small_stackoverflow = small_stackoverflow['Tags']
small_stackoverflow.head()

In [ ]:
semantic_technologies = pd.DataFrame(index=technical_tags,columns=technical_tags)
semantic_technologies = semantic_technologies.fillna(0)
semantic_technologies.head()

In [ ]:
for line in small_stackoverflow:
    currlist = line[1:-1].split('><')
    for x in currlist:
        for y in currlist:
            if ((x in technical_tags) and (y in technical_tags)):
                semantic_technologies.set_value(x, y, semantic_technologies.at[x, y] + 1)
                
semantic_tech_axis_sum0 = semantic_technologies.sum(axis=0)
semantic_tech_axis_sum1 = semantic_technologies.sum(axis=1)
# semantic_technologies.tail()
semantic_tech_axis_sum1.head()


for index, row in semantic_technologies.iterrows():
#     print(type(index))
    semantic_technologies.loc[index] = semantic_technologies.loc[index] /\
    (np.minimum(semantic_tech_axis_sum1[index], semantic_tech_axis_sum0))
    semantic_technologies.loc[index, index] = -1.0
semantic_technologies.fillna(0,inplace=True)

for x in technical_tags:
    print('For', x, 'popular tag is', np.argmax(semantic_technologies[x]))
# np.argmax(semantic_technologies['html'])
# semantic_technologies.loc['html', 'positioning']

# print(sorted(list(semantic_technologies['html'])))

# Кастомный датасет

In [ ]:
custom_small_set = pd.read_csv('small_custom_dataset.csv', sep=';')
custom_small_set

# Общая структура происходящего

In [ ]:
data = custom_small_set

#### TODO

In [ ]:
def geo_distance_metrics(geo1, geo2):
       return abs(geo1 - geo2)

In [ ]:
def geo_close_enough(geo1, geo2):
    return geo_distance_metrics(geo1, geo2) < 3000

In [ ]:
def closest_people(index):
    person = data.loc[index]
    
    print("Out req person:", person)
    
    stud_lambd1 = lambda x: \
(geo_close_enough(person.geo, x.geo) or (person.ementoring >= 1 and x.ementoring >= 1)) and \
x.user_role == 'mentor'

    ment_lambd1 = lambda x: \
(geo_close_enough(person.geo, x.geo) or (person.ementoring >= 1 and x.ementoring >= 1)) and \
x.user_role == 'student'
    
    if person.user_role == 'student':
        closest_people = data[data.apply(stud_lambd1, axis=1)]
        return closest_people
    elif person.user_role == 'mentor':
        closest_people = data[data.apply(ment_lambd1, axis=1)]
        return closest_people
        
#     return data.loc[index]

#### Жесткое косинусное расстояние

In [ ]:
def get_cosine(vec1, vec2):
    print('vec', vec1, vec2)
    
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

def list_to_vector(text):
    return Counter(text)

#### Мягкое косинусное расстояние

In [ ]:
def tech_tag_sim(w1, w2):
    if (w1 == w2):
        return 1.0
    elif (w1 in technical_tags and w2 in technical_tags):
        return semantic_technologies.loc[w1, w2]
    else:
        return 0.0
        

In [ ]:
def get_cosine_soft(vec1, vec2):
    print('vec', vec1, vec2)
    
    S1 = 0
    S2 = 0
    S3 = 0
    for x in vec1:
        for y in vec2:
            S1 += tech_tag_sim(x, y)
    for x in vec1:
        for y in vec1:
            S2 += tech_tag_sim(x, y)
    for x in vec2:
        for y in vec2:
            S3 += tech_tag_sim(x, y)
    
    S2 = S2**0.5
    S3 = S3**0.5
    
    if (S2 * S3 > 0):
        return S1 / (S2 * S3)
    else:
        return 0.0
#     intersection = set(vec1.keys()) & set(vec2.keys())
    
    
#     numerator = sum([vec1[x] * vec2[x] for x in intersection])

#     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
#     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
#     denominator = math.sqrt(sum1) * math.sqrt(sum2)

#     if not denominator:
#         return 0.0
#     else:
#         return float(numerator) / denominator

In [ ]:
get_cosine_soft(['c++'], ['c++'])

In [ ]:
get_cosine_soft(['html'], ['css'])

In [ ]:
get_cosine_soft(['c++'], ['animals'])

https://stackoverflow.com/questions/15173225/calculate-cosine-similarity-given-2-sentence-strings

In [ ]:
def check_person(person, x):
    w0 = 0
    w1 = 1
    xskills = [technical_tags_syn_try_replace(x) for x in x.skills_tags.split(',')] #x.skills_tags.replace(",", " ")
    xintro = x.intro_phrase_tags.split(',') # x.intro_phrase_tags.replace(",", " ")#
    
    personskills = [technical_tags_syn_try_replace(x) for x in person.skills_tags.split(',')] #person.skills_tags.replace(",", " ")
    personintro = person.intro_phrase_tags.split(',')#person.intro_phrase_tags.replace(",", " ")#
    
    x_alltags = xskills + xintro
    person_alltags = personskills + personintro
    
    xtechnical = [x for x in x_alltags if x in technical_tags]
    persontechnical = [x for x in person_alltags if x in technical_tags]
    xhuman = [x for x in x_alltags if not (x in technical_tags)]
    personhuman = [x for x in person_alltags if (not x in technical_tags)]

    return w0 * sentence_similarity(xhuman, personhuman) +\
        w1 * get_cosine_soft(list_to_vector(xtechnical), list_to_vector(persontechnical))

In [ ]:
def process_req(index):
    person = data.loc[index]
    
    closest1 = closest_people(index)
    
    sLength = len(closest1['id'])
    closest1['tag_score'] = closest1.apply(functools.partial(check_person, person), axis=1)
    return closest1.sort_values(by=['tag_score'], ascending=False)

In [ ]:
process_req(2)

# Эксперименты

In [ ]:
get_cosine(list_to_vector(['c++','java']), list_to_vector(['java','c#']))

In [ ]:
import numpy as np

In [ ]:
np.array([(1, 2),(1, 2),(1, 2)]).shape

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.DataFrame({'fieldA' : ['lol', 'kek', 'fuf'], 'fieldB' : ['anus1', 'anus2', 'anus1']})

In [ ]:
df1

In [ ]:
df2 = pd.DataFrame({'fieldB' : ['anus1', 'anus2'], 'fieldC' : [100, 200]})
df2

In [ ]:
pd.merge(df1, df2, on='fieldB', how='inner')

In [ ]:
df2 = df2.rename(index=str, columns={"fieldB": "whore"})
df2

In [ ]:
nltk.download()